# Feature Engineering

In [1]:
# distance to nearest L station or metra station
# distance to nearest Divvy station
# distance to nearest grocery store
# number of grocery stores within 10 minute walk

In [2]:
import pandas as pd
import numpy as np
import os
import geopy.distance
from EDA import map_functions

In [3]:
# config variables
ROOT = "C:/Users/Beau/Documents/GitHub/RealEstate"
SERIALIZED = os.path.join(ROOT, "data", "serialized")
CLEANED = os.path.join(ROOT, "data", "processed")
WALK_DISTANCE = 0.5

In [19]:
# read in datasets
# TODO: reset the index in the EDA files
# df_permits = pd.read_pickle(os.path.join(CLEANED, "Building_Permits.pkl")).reset_index()
# df_licenses = pd.read_pickle(os.path.join(CLEANED, "Business_Licenses.pkl")).reset_index()
# df_licenses = pd.read_pickle(os.path.join(CLEANED, "Business_Licenses_Current.pkl")).reset_index()
# df_licenses_liq = pd.read_pickle(os.path.join(DATA_DIR, "Business_Licenses_-_Current_Liquor_and_Public_Places_of_Amusement_Licenses.pkl")).reset_index()
# TODO: check if this is just a subset of the df_licenses
# df_crimes = pd.read_pickle(os.path.join(DATA_DIR, "Crimes_-_2001_to_Present.pkl")).reset_index()
df_grocery = pd.read_pickle(os.path.join(CLEANED, "Grocery.pkl")).reset_index()
# WARNING: df_grocery is slightly out of date (last updated 2020 I think?)
# df_landmarks = pd.read_pickle(os.path.join(CLEANED, "Individual_Landmarks.pkl")).reset_index()
df_murals = pd.read_pickle(os.path.join(CLEANED, "Murals.pkl")).reset_index()
df_art = pd.read_pickle(os.path.join(CLEANED, "Park_Art.pkl")).reset_index()
# TODO: population density from census
# TODO: employment growth from census?
# TODO: Divvy stations

df_listings = pd.read_pickle(os.path.join(CLEANED, "Redfin_Listings.pkl")).reset_index()
# TODO: get data for properties sold to go back in time and have a proper training set with ground truth (not listing price)

In [20]:
# get closest distance metrics
# df_listings["min_dist_to_permit"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_permits.LATITUDE, df_permits.LONGITUDE)
# df_listings["min_dist_to_license"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_licenses.LATITUDE, df_licenses.LONGITUDE)
df_listings["min_dist_to_grocery"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_grocery.LATITUDE, df_grocery.LONGITUDE)
# df_listings["min_dist_to_landmark"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_landmarks.LATITUDE, df_landmarks.LONGITUDE)
df_listings["min_dist_to_mural"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_murals.LATITUDE, df_murals.LONGITUDE)
df_listings["min_dist_to_art"] = map_functions.get_closest(df_listings.LATITUDE, df_listings.LONGITUDE, df_art.LATITUDE, df_art.LONGITUDE)


In [21]:
df_listings

,index,PROPERTY TYPE,ADDRESS,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,...,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),MLS#,LATITUDE,LONGITUDE,min_dist_to_grocery,min_dist_to_mural,min_dist_to_art
0,0,Condo/Co-op,3001 W Logan Blvd #1,60647,312900,2.0,1.0,CHI - Logan Square,950.0,NaN,...,5,329.0,338.0,https://www.redfin.com/IL/Chicago/3001-W-Logan...,11689023,41.927859,-87.702645,0.459995,0.324022,0.675326
1,1,Condo/Co-op,317 E 25th Pl Unit 3W,60616,290000,2.0,2.0,CHI - Near South Side,1214.0,NaN,...,5,239.0,335.0,https://www.redfin.com/IL/Chicago/317-E-25th-S...,11687898,41.847339,-87.619935,0.718396,0.730189,0.573372
2,2,Multi-Family (2-4 Unit),7847 S Saint Lawrence Ave,60619,225000,4.0,2.0,CHI - Greater Grand Crossing,NaN,NaN,...,5,NaN,NaN,https://www.redfin.com/IL/Chicago/7847-S-St-La...,11688895,41.751655,-87.609821,0.447490,1.123216,2.167356
3,3,Condo/Co-op,933 W Van Buren St #324,60607,290000,1.0,1.0,CHI - Near West Side,800.0,NaN,...,5,363.0,360.0,https://www.redfin.com/IL/Chicago/933-W-Van-Bu...,11686799,41.876095,-87.651260,0.223117,0.842772,0.449744
4,4,Single Family Residential,19 E 102nd Pl,60628,130000,3.0,2.0,CHI - Roseland,1633.0,4687.0,...,5,80.0,NaN,https://www.redfin.com/IL/Chicago/19-E-102nd-P...,11688879,41.707893,-87.622745,0.444903,1.218160,2.617911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,344,Condo/Co-op,6025 S Saint Lawrence Ave #2,60637,179000,3.0,2.0,CHI - Woodlawn,NaN,NaN,...,6,NaN,220.0,https://www.redfin.com/IL/Chicago/6025-S-St-La...,11688522,41.785037,-87.610586,0.237449,0.371764,0.189429
306,345,Single Family Residential,5831 W Newport Ave,60634,599000,5.0,3.5,CHI - Portage Park,2668.0,3781.0,...,6,225.0,NaN,https://www.redfin.com/IL/Chicago/5831-W-Newpo...,11688523,41.942848,-87.772784,0.386474,0.961685,0.862762
307,346,Condo/Co-op,4158 W Cullom Ave,60641,795000,3.0,3.5,CHI - Irving Park,NaN,NaN,...,6,NaN,288.0,https://www.redfin.com/IL/Chicago/4158-W-Cullo...,11688517,41.959507,-87.731724,0.558228,0.556612,1.101335
308,347,Townhouse,1211 W Lexington St,60607,600000,3.0,3.0,Columbus on the Park,2400.0,NaN,...,6,250.0,300.0,https://www.redfin.com/IL/Chicago/1211-W-Lexin...,11688514,41.872068,-87.657340,0.419165,0.741022,0.208722


In [22]:
# get count within distance metrics
# TODO: change distance for permits, licenses?
# TODO: need to filter out permits, licesnses because there are way too many
# df_listings[f"count_permits_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_permits.LATITUDE, df_permits.LONGITUDE, WALK_DISTANCE)
# df_listings[f"count_licenses_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_licenses.LATITUDE, df_licenses.LONGITUDE, WALK_DISTANCE)
df_listings[f"count_grocery_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_grocery.LATITUDE, df_grocery.LONGITUDE, WALK_DISTANCE)
# df_listings[f"count_landmarks_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_landmarks.LATITUDE, df_landmarks.LONGITUDE, WALK_DISTANCE)
df_listings[f"count_murals_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_murals.LATITUDE, df_murals.LONGITUDE, WALK_DISTANCE)
df_listings[f"count_art_{WALK_DISTANCE}_mi"] = map_functions.count_points_within_range(df_listings.LATITUDE, df_listings.LONGITUDE, df_art.LATITUDE, df_art.LONGITUDE, WALK_DISTANCE)


In [24]:
df_listings.to_pickle(os.path.join(CLEANED, "input.pkl"))